In [1]:
import sys
sys.path.append('../..')
import pybeamtools.controls as pc

[INFO ] [     MainThread] [15:14:21      logging.py  23] [pybeamtools.utils.logging] Starting shared logging thread on PID 16664
[INFO ] [   log_listener] [15:14:21      logging.py  32] [pybeamtools.utils.logging] Logging listener started on PID 16664
[INFO ] [     MainThread] [15:14:21      logging.py  27] [pybeamtools.utils.logging] Logging setup finished
[INFO ] [     MainThread] [15:14:22        utils.py 148] [numexpr.utils] Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[INFO ] [     MainThread] [15:14:22        utils.py 160] [numexpr.utils] NumExpr defaulting to 8 threads.
[DEBUG] [     MainThread] [15:14:23     _backend.py  35] [caproto] Backend 'array' registered
[DEBUG] [     MainThread] [15:14:23     _backend.py  35] [caproto] Backend 'numpy' registered
[DEBUG] [     MainThread] [15:14:23     _backend.py  47] [caproto] Selecting backend: 'numpy'


In [2]:
from pybeamtools.sim.core import SimulationEngine, ChannelMap, ChannelMapper
from pybeamtools.sim.devices import RealisticMagnet
mag = RealisticMagnet(name='magneto', value=0, low=-2, high=2, noise=0.1, resolution=None, model='instant')
sim = SimulationEngine()
sim.add_device(mag, period=5.0)
#sub = sim.subscribe_device(mag.name)
# def f(sub, v):
#     pass
# sub.add_callback(f)

def get_mag(device, output):
    return device.read()
def get_mag_exact(device, output):
    return device.read_exact()
def set_mag(device, output, value):
    return device.write(value)
m = ChannelMap(device=mag, output='TEST:CHANNEL', read_fun=get_mag, write_fun=set_mag)
m2 = ChannelMap(device=mag, output='TEST:CHANNEL:EXACT', read_fun=get_mag_exact, write_fun=set_mag)
chm = ChannelMapper(maps=[m, m2])
sim.add_mapper(chm)

sim.start_update_thread()

[DEBUG] [     MainThread] [15:14:23         core.py 135] [SimulationEngine] Added device (magneto) with update period of (5.0)
[DEBUG] [     MainThread] [15:14:23         core.py 151] [SimulationEngine] Added mapper for outputs (['TEST:CHANNEL:EXACT', 'TEST:CHANNEL']) from devices (['magneto'])
[DEBUG] [     MainThread] [15:14:23         core.py 207] [SimulationEngine] Starting poll thread
[DEBUG] [sim_engine_poll] [15:14:23         core.py 175] [SimulationEngine] Hello from simulation poll thread (id 2748)


In [3]:
from pybeamtools.controls.control_lib import ConnectionOptions
ao = pc.AcceleratorOptions(connection_settings=ConnectionOptions(network='dummy'))

In [4]:
acc = pc.Accelerator(options=ao,ctx=sim)

[INFO ] [     MainThread] [15:14:23  control_lib.py  49] [Accelerator] Control lib init
[INFO ] [     MainThread] [15:14:23      network.py 157] [SimConnectionManager] Creating dummy connection manager
[INFO ] [     MainThread] [15:14:23  control_lib.py  71] [Accelerator] Startup finished


In [5]:
# Add PVs
from pybeamtools.controls.network import SimPV, PVOptions, PVAccess
pv_settings = PVOptions(name='TEST:CHANNEL', low=0.0, high=5.0, security=PVAccess.READWRITE)
pv = SimPV(pv_settings)
pv_exact = SimPV(options=PVOptions(name='TEST:CHANNEL:EXACT', low=0.0, high=5.0, security=PVAccess.READWRITE))
pv, pv_exact

(SimPV name='TEST:CHANNEL' low=0.0 high=5.0 monitor=True security=<PVAccess.READWRITE: 3> read_timeout=2.0 write_timeout=5.0,
 SimPV name='TEST:CHANNEL:EXACT' low=0.0 high=5.0 monitor=True security=<PVAccess.READWRITE: 3> read_timeout=2.0 write_timeout=5.0)

In [6]:
sim.channels

['TEST:CHANNEL:EXACT', 'TEST:CHANNEL']

In [7]:
acc.add_pv_object([pv, pv_exact])
print

[DEBUG] [     MainThread] [15:14:23      network.py 184] [SimConnectionManager] Adding 2 PV objects
[DEBUG] [     MainThread] [15:14:23         core.py 164] [SimulationEngine] Created subscription for channel TEST:CHANNEL
[DEBUG] [     MainThread] [15:14:23         core.py 164] [SimulationEngine] Created subscription for channel TEST:CHANNEL:EXACT


<function print>

In [8]:
acc.cm.sim.subscriptions, acc.cm.sim.device_channels

({}, {'magneto': ['TEST:CHANNEL:EXACT', 'TEST:CHANNEL']})

In [9]:
acc.cm.sim.channel_subs

{'TEST:CHANNEL': <pybeamtools.sim.core.ChannelSubscription at 0x200df2d50a0>,
 'TEST:CHANNEL:EXACT': <pybeamtools.sim.core.ChannelSubscription at 0x200f876aa60>}

In [ ]:
pv

In [ ]:
pv.read()

In [ ]:
pv_exact.read()

In [ ]:
pv.write(2.0)
print()

In [ ]:
mag.read(), mag.read_exact()